# Density inversion
Most of *Invert4Geom's* functionality is centered on performing geometric inversions, to estimated the shape of a layer. However, we also support density inversions, there the density of each model element (prism or tesseroid) is updated to minimize the gravity misfit. To showcase this, we will create a synthetic observed gravity dataset calculated from the forward gravity of a  prism layer with spatially variable density values. We will then try and recover these density values during the inversion. 

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import verde as vd
from polartoolkit import maps
from polartoolkit import utils as polar_utils

import invert4geom

os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

## Create observed gravity data
To run the inversion, we need to have observed gravity data. In this simple example, we will first create a synthetic topography, which represents the `true` Earth topography which we hope to recover during the inversion. From this topography, we will create a layer of vertical right-rectangular prisms, which allows us to calculated the gravity effect of the topography. This will act as our observed gravity data. 

In [ ]:
# load bishop model
bishop_model = invert4geom.load_bishop_model(coarsen_factor=50)

# get the basement topography
topography = invert4geom.utils.normalize_xarray(
    bishop_model.basement_topo, low=-5000, high=-10
)

spacing, region, _, _, _ = polar_utils.get_grid_info(topography)

fig = maps.plot_grd(
    topography,
    fig_height=10,
    title="Basement topography",
    cmap="viridis",
    hist=True,
    cbar_label="elevation (m)",
    frame=["nSWe", "xaf10000", "yaf10000"],
)

# scale between 1200 and 2400 kg/m³ to use as sediment density values
true_density = invert4geom.utils.normalize_xarray(
    bishop_model.moho_topo, low=1200, high=2400
)
# plot the true density
fig = maps.plot_grd(
    true_density,
    fig=fig,
    origin_shift="x",
    fig_height=10,
    title="True density",
    cmap="viridis",
    hist=True,
    cbar_label="density (kg/m³)",
    frame=["nSwe", "xaf10000", "yaf10000"],
)
fig.show()

### Prism layer

In [ ]:
model = invert4geom.create_model(
    zref=0,
    density_contrast=true_density,
    topography=topography.to_dataset(name="upward"),
)

In [ ]:
model.inv.plot_model(
    color_by="density",
    zscale=20,
)

### Forward gravity of prism layer

In [ ]:
# make pandas dataframe of locations to calculate gravity
# this represents the station locations of a gravity survey
# create lists of coordinates
coords = vd.grid_coordinates(
    region=region,
    spacing=spacing,
    pixel_register=False,
    extra_coords=100,  # survey elevation
)

# grid the coordinates
observations = vd.make_xarray_grid(
    (coords[0], coords[1]),
    data=coords[2],
    data_names="upward",
    dims=("northing", "easting"),
)

grav_data = invert4geom.create_data(observations)

grav_data.inv.forward_gravity(model, "gravity_anomaly")
grav_data

In [ ]:
# plot the observed gravity
fig = maps.plot_grd(
    grav_data.gravity_anomaly,
    fig_height=10,
    title="Observed gravity",
    cmap="viridis",
    hist=True,
    cbar_label="mGal",
    frame=["nSWe", "xaf10000", "yaf10000"],
)
fig.show()

## Gravity misfit
Now we need to create a starting model of the topography to start the inversion with. Since here we have no knowledge of either the topography or the appropriate reference level (`zref`), the starting model is flat, and therefore it's forward gravity is 0. If you had a non-flat starting model, you would need to calculate it's forward gravity effect, and subtract it from our observed gravity to get a starting gravity misfit. 

In this simple case, we assume that we know the true density contrast and appropriate reference value for the topography (`zref`), and use these values to create our flat starting model. Note that in a real world scenario, these would be unknowns which would need to be carefully chosen, as explained in the following notebooks.

In [ ]:
model = invert4geom.create_model(
    zref=0,
    density_contrast=1800,
    topography=topography.to_dataset(name="upward"),
)
model

In [ ]:
grav_data.inv.forward_gravity(
    model,
    progressbar=True,
)

In [ ]:
# in many cases, we want to remove a regional signal from the misfit to isolate the
# residual signal. In this simple case, we assume there is no regional misfit and set
# it to 0
grav_data.inv.regional_separation(
    method="constant",
    constant=0,
)
grav_data

In [ ]:
grav_data.inv.plot_anomalies()

## Perform inversion
Now that we have a starting model and residual gravity misfit data we can start the inversion.

In [ ]:
# setup the inversion
inv = invert4geom.Inversion(
    grav_data,
    model,
    style="density",
    deriv_type="finite_difference",
    # set stopping criteria
    max_iterations=1,
)

# run the inversion
inv.invert()

In [ ]:
inv.stats_df

In [ ]:
inv.plot_inversion_results(
    iters_to_plot=1,
)

In [ ]:
_ = polar_utils.grd_compare(
    true_density,
    inv.model.density_contrast,
    grid1_name="True density",
    grid2_name="Inverted density",
    robust=True,
    hist=True,
    inset=False,
    verbose="q",
    title="difference",
)